In [18]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import datetime as dt

In [19]:
url = "https://www.genie.co.kr/chart/top200?ditc=D&rtm=N"

userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"

session = requests.Session()

session.headers.update({
        "Referer": "",
        "User-Agent": userAgent,
})

r = session.get(url)

if r.status_code != 200:
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    raise Exception(msg)

r.encoding = "utf-8"
soup = BeautifulSoup(r.text)
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!--[if lte IE 6]><html dir="ltr" class="no-js ie6"><![endif]-->
<!--[if lte IE 7]><html dir="ltr" class="no-js ie7"><![endif]-->
<!--[if IE 8]><html dir="ltr" class="no-js ie8"><![endif]-->
<!--[if IE 9]><html dir="ltr" class="no-js ie9"><![endif]-->
<!--[if (gte IE 10)|!(IE)]><!--><html class="no-js" dir="ltr"><!--<![endif]-->
<title>음악, 그리고 설레임 - 지니</title>
<meta charset="utf-8"/>
<meta content="d17289f97c7f294985a9b0eb3649a45ac5ceeee2" name="naver-site-verification"/>
<meta content="-l7MNwwsDcn29Pnu00OagSrpDUPbjqLYoY08rhXLde8" name="google-site-verification"/>
<meta content="가격은 가볍게, 추천은 확실하게! 최저가로 즐기는 프리미어 사운드! 국내 최초 실시간 TOP 200 차트" name="description"/>
<meta content="width=1200" name="viewport"/>
<meta content="" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="http://image.genie.co.kr" property="og:image">
<meta content="https://image.genie.co.kr" prope

In [20]:
# 오늘 날짜 구하기
today = dt.date.today()

# 이번 달 1일 날짜 구하기
start_month = dt.date(today.year, today.month, 1)

# 어제 날짜 구하기
yesterday = today - dt.timedelta(days=1)

In [21]:
# 수집할 음원 사이트의 주소 페이지 설정
# 1~50, 51~100으로 2개의 페이지로 나눠져 있음
pages = ['1', '2']

# dict 형식의 노래제목, 가수, 앨범, 순위를 담을 빈 list 생성
genie_chart = []

# 날짜별 데이터 수집(이번 달 1일 부터 어제 날짜까지)
for i in range((yesterday - start_month).days + 1):
    # 날짜 계산
    date = (start_month + dt.timedelta(days=i)).strftime("%Y%m%d")
    
    
    for p in pages:
        # 수집할 컨텐츠가 있는 날짜별 웹 페이지 주소
        url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={date}&hh=09&rtm=N&pg={p}'

        # 음원 정보 추출
        song_elements = soup.select('.title.ellipsis')
        artist_elements = soup.select('.artist.ellipsis')
        album_elements = soup.select('.albumtitle.ellipsis')

        # 추출한 정보를 리스트에 추가
        for i in range(len(song_elements)):
            song = song_elements[i].text.strip()
            singer = artist_elements[i].text.strip()
            album = album_elements[i].text.strip()

            rank = (int(p) - 1) * 50 + (i + 1)

            item = {"가수": singer, "노래 제목": song, "앨범": album, "순위": rank}
            genie_chart.append(item)
genie_chart

[{'가수': '(여자)아이들', '노래 제목': '퀸카 (Queencard)', '앨범': 'I feel', '순위': 1},
 {'가수': 'IVE (아이브)', '노래 제목': 'I AM', '앨범': "I've IVE", '순위': 2},
 {'가수': 'aespa',
  '노래 제목': 'Spicy',
  '앨범': 'MY WORLD - The 3rd Mini Album',
  '순위': 3},
 {'가수': 'LE SSERAFIM (르세라핌)',
  '노래 제목': '이브, 프시케 그리고 푸른 수염의 아내',
  '앨범': 'UNFORGIVEN',
  '순위': 4},
 {'가수': 'IVE (아이브)', '노래 제목': 'Kitsch', '앨범': "I've IVE", '순위': 5},
 {'가수': 'LE SSERAFIM (르세라핌)',
  '노래 제목': 'UNFORGIVEN (Feat. Nile Rodgers)',
  '앨범': 'UNFORGIVEN',
  '순위': 6},
 {'가수': 'NewJeans',
  '노래 제목': 'Hype boy',
  '앨범': "NewJeans 1st EP 'New Jeans'",
  '순위': 7},
 {'가수': '박재정', '노래 제목': '헤어지자 말해요', '앨범': '1집 Alone', '순위': 8},
 {'가수': 'NewJeans', '노래 제목': 'Ditto', '앨범': "NewJeans 'OMG'", '순위': 9},
 {'가수': '윤하 (YOUNHA)',
  '노래 제목': '사건의 지평선',
  '앨범': "YOUNHA 6th Album Repackage 'END THEORY : Final Edition'",
  '순위': 10},
 {'가수': '지수 (JISOO)', '노래 제목': '꽃', '앨범': 'ME', '순위': 11},
 {'가수': 'Charlie Puth',
  '노래 제목': "I Don't Think That I Like Her",
  '앨범': 'CHA

In [22]:
# 결과를 데이터프레임으로 변환
df = DataFrame(genie_chart)
df.set_index('순위', inplace=True)
# 결과를 엑셀 파일로 저장
df.to_excel("확인용.xlsx")